In [186]:
import pandas as pd

In [187]:
df = pd.read_excel('Data/2018/Oferta_de_empleo_Costa_Rica_4_trimestre_2018.xlsx', sheet_name=1)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Cuadro 1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Costa Rica: Compendio del IV trimestre de 2018...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Características del empleo,IV Trimestre 2018,NaN,NaN,NaN,NaN
5,NaN,NaN,Población ocupada por sexo,NaN,Ingreso bruto medio en el empleo principal por...,NaN,Brecha del ingreso2/
6,NaN,NaN,Hombre,Mujer,Hombre,Mujer,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,1. Rama de actividad,1332961,832362,467947.25194,405666.402888,86.690626
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
df.drop(df.columns[0], axis=1, inplace=True)

In [189]:
df.isna().sum()

Unnamed: 1    15
Unnamed: 2    25
Unnamed: 3    27
Unnamed: 4    26
Unnamed: 5    27
Unnamed: 6    27
dtype: int64

In [190]:
df.dropna(how='all', inplace=True)

In [191]:
data_period = df['Unnamed: 2'][4]
df.drop([0,2,4], axis=0, inplace=True)

In [192]:
column_names = {'Unnamed: 1': 'Sector industrial',
                'Unnamed: 2': 'Población ocupada por sexo: Hombre',
                'Unnamed: 3': 'Población ocupada por sexo: Mujer',
                'Unnamed: 4': 'Ingreso bruto medio en el empleo principal por sexo: Hombre',
                'Unnamed: 5': 'Ingreso bruto medio en el empleo principal por sexo: Mujer',
                'Unnamed: 6': 'brecha del ingreso'}

df.rename(columns=column_names, inplace=True)
total_trabajadores = df.loc[8:8].reset_index(drop=True)
df.drop([5,6,8], axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [193]:
total_sector = df[df['Sector industrial'].str.contains('Sector|1.4 No especificado')].reset_index(drop=True)
sector_publico_privado = df[df['Sector industrial'].str.contains('Público|Privado')].reset_index(drop=True)
Posición_empleo = df[df['Sector industrial'].str.contains('Asalariada|Cuenta propia|Empleadora|Auxiliar no remunerado')].reset_index(drop=True)

In [194]:
df.drop(df[df['Sector industrial'].str.contains('Sector')].index, inplace=True)
df = df.drop(df.loc[16:39].index).reset_index(drop=True)

In [196]:
df

,Sector industrial,Población ocupada por sexo: Hombre,Población ocupada por sexo: Mujer,Ingreso bruto medio en el empleo principal por sexo: Hombre,Ingreso bruto medio en el empleo principal por sexo: Mujer,brecha del ingreso
0,"Agricultura, ganadería y pesca",229512,36759,334316.274224,374773.703244,112.101543
1,Industria manufacturera,154018,83006,419304.673749,336096.435425,80.155662
2,Construcción,137657,2424,358193.359928,348158.226811,97.198403
3,Otros3/,24568,2375,870981.400962,557126.346316,63.965355
4,Comercio y reparación,217753,149391,409698.678646,263725.282086,64.370547
5,Transporte y almacenamiento,97429,12185,370475.596764,562934.632734,151.94918
6,Hoteles y restaurantes,56809,74162,410051.21817,252698.11445,61.625988
7,Intermediación financiera y de seguros,27208,18415,875702.2645,787346.105774,89.910251
8,Actividades profesionales y administrativas de...,140016,63611,426446.968007,446988.445957,104.816889
9,Administración pública,52331,33875,890258.586599,1057642.172422,118.801682
